In [2]:
import pandas as pd
import json
import numpy as np
import pickle
import ast

pd.set_option('display.max_columns', None)

ETL de Archivos YELP

1- Archivo business.pkl

Leemos el archivo

In [3]:
df_business= pd.read_pickle('business.pkl')

In [4]:
# Eliminar las columnas duplicadas con el mismo nombre
df_business = df_business.loc[:,~df_business.columns.duplicated()]

In [5]:
# Convertimos a númerico la columna 'postal_code'
df_business["postal_code"] = df_business["postal_code"].apply(pd.to_numeric, errors='coerce')

In [6]:
# Filtrar las filas que contengan "Restaurants" en la columna 'categories'
df_business = df_business[df_business['categories'].str.contains('Restaurants', na=False)]

In [8]:
df_business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107.0,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,AZ,37015.0,36.269593,-87.058943,2.0,6,1,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '..."
8,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,TN,63123.0,38.565165,-90.321087,3.0,19,0,"{'Caters': 'True', 'Alcohol': 'u'full_bar'', '...","Pubs, Restaurants, Italian, Bars, American (Tr...",None
9,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,MO,37207.0,36.208102,-86.76817,1.5,10,1,"{'RestaurantsAttire': ''casual'', 'Restaurants...","Ice Cream & Frozen Yogurt, Fast Food, Burgers,...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-21:0', '..."
11,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,Tampa Bay,MO,33602.0,27.955269,-82.45632,4.0,10,1,"{'Alcohol': ''none'', 'OutdoorSeating': 'None'...","Vietnamese, Food, Restaurants, Food Trucks","{'Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'..."


In [26]:
#df_business.to_csv('C:/Users/cesua/OneDrive/Documentos/Henry/Proyecto_Grupal_GoogleMaps_Yelp/Datasets/business.csv', index=False)

2- Archivo 'review.json'

Leemos el archivo review.json de Yelp

In [3]:
jsonarr=[]
with open("review.json", 'r',encoding='utf-8') as f:
    for line in f:
        jsonarr.append(json.loads(line))


df_review = pd.DataFrame(jsonarr)

In [4]:
# Eliminamos duplicados y reindexamos
df_review= df_review.drop_duplicates(subset=["review_id", "text"], keep="first")
df_review['id_review'] = df_review.index

In [5]:
df_review['date'] = pd.to_datetime(df_review['date'])
df_review['fecha'] = df_review['date'].dt.date
df_review['hora'] = df_review['date'].dt.time

# Extraer el año en una columna separada
df_review['año'] = df_review['date'].dt.year


In [8]:
filtered_df = df_review[df_review['año'].isin([2020, 2021, 2022, 2023])]

In [21]:
# Eliminar la columna original 'date' si ya no la necesitas
#filtered_df.drop('date', axis=1, inplace=True)
filtered_df.drop('hora', axis=1, inplace=True)
filtered_df.drop('id_review', axis=1, inplace=True)

C:\Users\cesua\AppData\Local\Temp\ipykernel_22564\3077628813.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.drop('hora', axis=1, inplace=True)
C:\Users\cesua\AppData\Local\Temp\ipykernel_22564\3077628813.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.drop('id_review', axis=1, inplace=True)


In [22]:
filtered_df.tail()

,review_id,user_id,business_id,stars,useful,funny,cool,text,fecha,año
6990268,MIXdpbbCTRAVdi8RiMjwdg,s67G457QlHSvk5RjOMN91w,58MJvmfo5hyfBbvkr54sFA,5.0,1,0,0,Great gym. Was in Indy for 4 days on business...,2022-01-18,2022
6990272,wD5ZWao_vjyT2h4xmGam8Q,7L7GL5Pi2cf8mbm2Dpw4zw,e_E-jq9mwm7wk75k7Yi-Xw,5.0,1,0,1,It is very rare for a restaurant to be this go...,2022-01-17,2022
6990273,zHZ-A1qyKDEgyZMDaD--wg,_XVdmFWSgTN6YlojUxixTA,6WaI-IN8ql0xpEKlb4q8tg,5.0,1,0,0,We redesigned my moms dress and mad it complet...,2022-01-17,2022
6990276,shTPgbgdwTHSuU67mGCmZQ,Zo0th2m8Ez4gLSbHftiQvg,2vLksaMmSEcGbjI5gywpZA,5.0,2,1,2,"This spot offers a great, affordable east week...",2021-03-31,2021
6990278,i-I4ZOhoX70Nw5H0FwrQUA,YwAMC-jvZ1fvEUum6QkEkw,Rr9kKArrMhSLVE9a53q-aA,5.0,1,0,0,For when I'm feeling like ignoring my calorie-...,2022-01-19,2022


In [23]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1204411 entries, 265179 to 6990278
Data columns (total 10 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   review_id    1204411 non-null  object 
 1   user_id      1204411 non-null  object 
 2   business_id  1204411 non-null  object 
 3   stars        1204411 non-null  float64
 4   useful       1204411 non-null  int64  
 5   funny        1204411 non-null  int64  
 6   cool         1204411 non-null  int64  
 7   text         1204411 non-null  object 
 8   fecha        1204411 non-null  object 
 9   año          1204411 non-null  int32  
dtypes: float64(1), int32(1), int64(3), object(5)
memory usage: 96.5+ MB


In [24]:
filtered_df.to_csv('C:/Users/cesua/OneDrive/Documentos/Henry/Proyecto_Grupal_GoogleMaps_Yelp/Datasets/review_restaurants.csv')

3- tip.json

Abrimos el archivo json

In [34]:
df_business2 = pd.read_csv('business.csv')

In [35]:
json_objects=[]

with open('tip.json', 'r',encoding='utf-8') as f:
    for line in f:
        json_objects.append(json.loads(line))


df_tip = pd.DataFrame(json_objects)

In [39]:
df_tip.head()

,user_id,business_id,text,date,compliment_count
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,2017-06-27 23:05:38,0
4,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,2012-10-06 19:43:09,0
5,trf3Qcz8qvCDKXiTgjUcEg,7Rm9Ba50bw23KTA8RedZYg,"Chili Cup + Single Cheeseburger with onion, pi...",2012-03-13 04:00:52,0


In [38]:
# Obtenemos un DataFrame que contenga solo las filas correspondientes a los negocios presentes en Business.csv

df_tip= df_tip[df_tip['business_id'].isin(df_business2.business_id.unique().tolist())]

In [40]:
df_tip.info()

<class 'pandas.core.frame.DataFrame'>
Index: 649150 entries, 1 to 908914
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   user_id           649150 non-null  object
 1   business_id       649150 non-null  object
 2   text              649150 non-null  object
 3   date              649150 non-null  object
 4   compliment_count  649150 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 29.7+ MB


In [41]:
df_tip.to_csv('C:/Users/cesua/OneDrive/Documentos/Henry/Proyecto_Grupal_GoogleMaps_Yelp/Datasets/tip.csv')